<a href="https://colab.research.google.com/github/hugomaharavo/ai4eng-20242-Pruebas-Saber-Pro-Colombia/blob/main/99%20-%20modelo%20soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ACCEDER A LOS ARCHIVOS EN KAGGLE
# 1.1 - Instale la biblioteca opendatasets para descargar conjuntos de datos directamente de Kaggle
!pip install opendatasets

# 1.2 - Importar los módulos necesarios
import opendatasets as kaggle_data
import os

# 1.3 - Enlace al concurso de Kaggle
competition_url = "https://www.kaggle.com/competitions/udea-ai4eng-20242"

# 1.4 - Descargar los datos del concurso
kaggle_data.download(competition_url)

# 1.5 - Cambiar de directorio para acceder a la carpeta del conjunto de datos descargado
project_dir = "udea-ai4eng-20242"
os.chdir(project_dir)

# 1.6 - Comprobación de los archivos descargados
files = os.listdir()
print("Fichiers disponibles dans le dossier : ", files)

#2865fd2678d42f2f4002a048b30039ea <--- API key

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: hugomaharavo
Your Kaggle Key: ··········


100%|██████████| 20.1M/20.1M [00:00<00:00, 74.7MB/s]



Extracting archive ./udea-ai4eng-20242/udea-ai4eng-20242.zip to ./udea-ai4eng-20242
Fichiers disponibles dans le dossier :  ['test.csv', 'train.csv', 'submission_example.csv']


In [3]:
# 2 - IMPORTAR BIBLIOTECAS
# Importación de las bibliotecas necesarias para el tratamiento de datos y la formación de modelos
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
import lightgbm as lgb
from sklearn.metrics import accuracy_score

# 3 - CARGAR DATOS INICIALES
# Carga de dataset train y test
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# 4 - APLICAR MAPPINGS Y CONVERSIONES
# Mapping et conversiones de valores categóricos en valores numéricos para un mejor procesamiento

# Conversión de valores ordinales en la columna 'ESTU_VALORMATRICULAUNIVERSIDAD
tuition_average_mapping = {
    "No pagó matrícula": 0.0, "Menos de 500 mil": 0.25, "Entre 500 mil y menos de 1 millón": 0.75,
    "Entre 1 millón y menos de 2.5 millones": 1.75, "Entre 2.5 millones y menos de 4 millones": 3.25,
    "Entre 4 millions et moins de 5.5 millions": 4.75, "Entre 5.5 millions et menos de 7 millones": 6.25, "Más de 7 millones": 7.5
}
train_df['ESTU_VALORMATRICULAUNIVERSIDAD'] = train_df['ESTU_VALORMATRICULAUNIVERSIDAD'].map(tuition_average_mapping)
test_df['ESTU_VALORMATRICULAUNIVERSIDAD'] = test_df['ESTU_VALORMATRICULAUNIVERSIDAD'].map(tuition_average_mapping)

# Conversión de valores ordinales en la columna 'FAMI_ESTRATOVIVIENDA'
housing_strata_mapping = {"Estrato 1": 1, "Estrato 2": 2, "Estrato 3": 3, "Estrato 4": 4, "Estrato 5": 5, "Estrato 6": 6, "Sin Estrato": None}
train_df['FAMI_ESTRATOVIVIENDA'] = train_df['FAMI_ESTRATOVIVIENDA'].map(housing_strata_mapping)
test_df['FAMI_ESTRATOVIVIENDA'] = test_df['FAMI_ESTRATOVIVIENDA'].map(housing_strata_mapping)

# Conversión de columnas binarias en 0 y 1 para las columnas 'FAMI_TIENEINTERNET' et 'ESTU_PAGOMATRICULAPROPIO'
binary_mappings = {"Si": 1, "No": 0}
train_df['FAMI_TIENEINTERNET'] = train_df['FAMI_TIENEINTERNET'].map(binary_mappings)
test_df['FAMI_TIENEINTERNET'] = test_df['FAMI_TIENEINTERNET'].map(binary_mappings)
train_df['ESTU_PAGOMATRICULAPROPIO'] = train_df['ESTU_PAGOMATRICULAPROPIO'].map(binary_mappings)
test_df['ESTU_PAGOMATRICULAPROPIO'] = test_df['ESTU_PAGOMATRICULAPROPIO'].map(binary_mappings)

# Codificación de la variable de destino 'RENDIMIENTO_GLOBAL' en valores numéricos
target_mapping = {'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3}
train_df['RENDIMIENTO_GLOBAL'] = train_df['RENDIMIENTO_GLOBAL'].map(target_mapping)

# 5 - SEPARACIÓN DE CARACTERÍSTICAS Y VARIABLE OBJETIVO
# Separación de características (X) y objetivo (y)
X_train = train_df.drop(columns=['ID', 'RENDIMIENTO_GLOBAL'])
y_train = train_df['RENDIMIENTO_GLOBAL']
X_test = test_df.drop(columns=['ID'])

# 6 - IDENTIFICACIÓN DE LAS COLUMNAS DE PRETRATAMIENTO
# Identificación de las columnas numéricas y categóricas de pretratamiento
numerical_cols = X_train.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = X_train.select_dtypes(include=['object']).columns

# 7 - PREPROCESAMIENTO PIPELINE
# Configuración del preprocesamiento para datos numéricos y categóricos
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),  # Imputación de valores numéricos perdidos
            ('scaler', StandardScaler())  # Normalización de datos numericos
        ]), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)  # Codificación One-Hot para columnas categóricas
    ]
)

# 8 - MODELO Y VALIDACIÓN
# Configuration du modèle LightGBM
model = lgb.LGBMClassifier(random_state=42)

# Creación de un pipeline completo que incluya el preprocesador y el modelo
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', model)
])

# Validación cruzada estratificada para evaluar el rendimiento del modelo
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring='accuracy')

# 9 - ENTRENAMIENTO DEL MODELO
# Entrenamiento del pipeline en el dataset train completo
pipeline.fit(X_train, y_train)

# 10 - HACER PREDICCIONES SOBRE EL CONJUNTO DE PRUEBAS
# Predicciones sobre el conjunto de pruebas
predictions = pipeline.predict(X_test)

# 11 - CONVERTIR LAS PREDICCIONES EN PALABRAS ORIGINALES
# Invertir la codificación de las clases para obtener las predicciones en su formato original.
target_inverse_mapping = {0: 'bajo', 1: 'medio-bajo', 2: 'medio-alto', 3: 'alto'}
predictions_labels = [target_inverse_mapping[pred] for pred in predictions]

# 12 - CREAR EL FICHERO DE PRESENTACIÓN
# Creación del fichero de envío con predicciones
submission_df = pd.DataFrame({
    "ID": test_df['ID'],
    "RENDIMIENTO_GLOBAL": predictions_labels
})
submission_df.to_csv("submission.csv", index=False)

# Confirmación de la creación del fichero de envío
print("Fichero submission.csv generado con éxito")
print("Dimensiones de la clasificación submission.csv (filas, columnas) ", submission_df.shape)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022392 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1690
[LightGBM] [Info] Number of data points in the train set: 554000, number of used features: 835
[LightGBM] [Info] Start training from score -1.387096
[LightGBM] [Info] Start training from score -1.391216
[LightGBM] [Info] Start training from score -1.395033
[LightGBM] [Info] Start training from score -1.371986
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.021133 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1694
[LightGBM] [Info] Number of data points in the train set: 554000, number of used features: 837
[LightGBM] [Info] Start training from score -1.387089
[LightGBM] [Info] St

In [4]:
# 13 - DESCARGAR EL ARCHIVO
from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>